
### Atividade Prática 3 - MongoDB

1. Extrair os dados da API do Banco Central (Atividades 3 e 4).

2. Criar um banco de dados no MongoDB Atlas e as respectivas 
coleções para armazenar os recursos retornados pela API.

3. Inserir no banco e nas coleções criadas os dados extraídos da API.

O MongoDB é um banco orientado a documentos, 
portanto os registros devem estar em formato JSON.


In [19]:
import requests
import json
import pymongo
import pandas as pd

In [20]:
from pymongo import MongoClient

### 1. Conectando ao MongoDB Atlas


1. Entrar no MongoDB Atlas;
2. Criar um Cluster gratuito (se ainda não tiver);
3. Clicar em Connect > Connect your application.

In [ ]:
# Colar aqui a string de conexão do Atlas: NÃO COMPARTILHE SEU USUÁRIO E SENHA
MONGO_URI = ""

# Criar conexão
client = MongoClient(MONGO_URI)

### 2. Definindo URL para requisição da API

In [22]:
url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaDia(moeda=@moeda,dataCotacao=@dataCotacao)?@moeda='USD'&@dataCotacao='03-06-2006'&$top=1000&$format=json&$select=paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"

### 3. Criando a função para fazer a requisição e salvar o JSON na pasta Datasets

In [23]:
def requisicao_api(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:      #sucesso na requisição
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('datasetsUSD/usdDia.json', 'w', encoding='utf-8') as arquivo:   #salvar arquivo json
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)

### 4. Executando a função para buscar os dados

In [24]:
requisicao_api (url)

Status Code: 200


### 5. Lendo o arquivo JSON salvo e transformando em DataFrame

In [25]:
jsondata = pd.read_json('datasetsUSD/usdDia.json')

dados_df = pd.json_normalize(jsondata['value'])

### 6. Visualizando as primeiras linhas do DataFrame

In [26]:
dados_df.head()

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1.0,1.0,2.113,2.123,2006-03-06 09:19:00.0,Abertura
1,1.0,1.0,2.121,2.124,2006-03-06 09:30:00.0,Intermediário
2,1.0,1.0,2.122,2.124,2006-03-06 10:00:00.0,Intermediário
3,1.0,1.0,2.121,2.123,2006-03-06 10:30:00.0,Intermediário
4,1.0,1.0,2.120,2.121,2006-03-06 11:00:00.0,Intermediário


### 7. Função para salvar o JSON numa Coleção MongoDB Atlas

In [27]:
db = client['db_cambio']  # criar banco de dados
colecao = db['cotacao_usd_dia']  # criar coleção

def salvar_mongodb(dados):
    registros = dados.to_dict(orient='records')  # converter DataFrame para lista de dicionários
    colecao.insert_many(registros)  # inserir múltiplos documentos na coleção
    print(f'{len(registros)} registros inseridos na coleção.')
salvar_mongodb(dados_df)



17 registros inseridos na coleção.
